**数据挖掘模型中的$IV$和$WOE$详解**

# 背景介绍

* 挑选入模变量过程是个比较复杂的过程，需要考虑的因素很多，比如：变量的预测能力，变量之间的相关性，变量的简单性（容易生成和使用），变量的强壮性（不容易被绕过），变量在业务上的可解释性（被挑战时可以解释的通）等等。但是，其中最主要和最直接的衡量标准是变量的**预测能力**。


* 可是如何使用可量化的指标衡量每自变量的预测能力，并根据这些量化指标的大小，来确定哪些变量进入模型呢？$IV$就是这样一种指标，他可以用来衡量自变量的预测能力。类似的指标还有信息增益、基尼系数等等。


* 在分类模型中（特别是$logistics$），对于连续型变量一般都要分箱处理（如年龄分为10-20，20-30等区间），而如何去分箱并且赋于什么值，是相对有技术含量的地方。一般而言，会用卡方值或者信息熵去作为分箱标准，而分箱后会赋于$woe$，有的会赋 $0$ $1$ $2$ 这类哑变量。


* 直观上可以这样理解“用$IV$去衡量变量预测能力”这件事情：我们假设在一个分类问题中，目标变量的类别有两类：$Y_1$，$Y_2$。对于一个待预测的个体$A$，要判断$A$属于$Y_1$还是$Y_2$，我们是需要一定的信息的，假设这个信息总量是$I$，而这些所需要的信息，就蕴含在所有的自变量$C_1$，$C_2$，$C_3$，……，$C_n$中，那么，对于其中的一个变量$C_i$来说，其蕴含的信息越多，那么它对于判断$A$属于$Y_1$还是$Y_2$的贡献就越大，$C_i$的信息价值就越大，$C_i$的$IV$就越大。


* 实际应用中，可以根据IV从高到低选择哪些变量入模

# $IV$值计算

筛选合适的自变量，并进行适当的编码，事实上就是挑选并构造出对目标变量有较高预测力(predictive power)的自变量。这就是$IV$值的价值。为了介绍$IV$的计算方法，我们首先需要认识和理解另一个概念——$WOE$，因为$IV$的计算是以$WOE$为基础的。

## $WOE$值

$WOE$的全称是“Weight of Evidence”，即证据权重。$WOE$是对原始自变量的一种编码形式。要对一个变量进行$WOE$编码，需要首先把这个变量进行分组处理（也叫离散化、分箱）。分组后，对于第$i$组，$WOE$的计算公式如下：

$$WOE_i=ln(\frac{py_i}{pn_i})=ln(\frac{\frac{\#y_i}{\#y_T}}{\frac{\#n_i}{\#n_T}})$$  

直观上理解，$WOE$表示的是“**当前分组中逾期客户占所有逾期客户的比例”和“当前分组中没有逾期的客户占所有没有逾期的客户的比例**”的差异。简单变换后

$$WOE_i=ln(\frac{py_i}{pn_i})=ln(\frac{\frac{\#y_i}{\#y_T}}{\frac{\#n_i}{\#n_T}})=ln(\frac{\frac{\#y_i}{\#n_i}}{\frac{\#y_T}{\#n_T}})$$ 

其中：

* $py_i$是这个组中逾期客户（风险模型中，对应的是违约客户，总之，指的是模型中预测变量取值为“是”或者说1的个体）占所有样本中所有逾期客户的比例

  * $\#y_i$是这个组中逾期客户的数量
  * $\#n_i$是这个组中未逾期客户的数量

* $pn_i$是这个组中未逾期客户占样本中所有未逾期客户的比例
  * $\#y_T$是样本中所有逾期客户的数量
  * $\#n_T$是样本中所有未逾期客户的数量。

## $IV$值

对于分组后的变量，第$i$组的$WOE$前面已经介绍过，对于分组$i$，也会有一个对应的$IV$值，即：
$$ IV_i=(py_i-pn_i)*WOE_i= (py_i-pn_i)*ln(\frac{py_i}{pn_i})=(\frac{\#y_i}{\#y_T}-\frac{\#n_i}{\#n_T})*ln(\frac{\frac{\#y_i}{\#y_T}}{\frac{\#n_i}{\#n_T}}) $$

其中：
* $\#y_i$是这个组中逾期客户的数量
* $\#y_T$是样本中所有逾期客户的数量


* $\#n_i$是这个组中未逾期客户的数量
* $\#n_T$是样本中所有未逾期客户的数量
  
  
  
变量的$IV$值相当于是该变量$WOE$值的一个加权求和，其值的大小决定了自变量对于目标变量的影响程度。如下
$$IV=\sum_{i=1}^nIV_i=\sum_{i=1}^n(py_i-pn_i)*WOE_i$$ 

其中$n$为变量分组个数。

## $WOE$和$IV$值的区别


> **$WOE$的基本特点**

  * 当前分组中，逾期的比例越大，$WOE$值越大；且$WOE$描述了变量当前分组对判断个体是否会逾期所起到的**影响方向**和**大小**。
  * 当前分组$WOE$的正负，由当前分组逾期和未逾期的比例与样本整体逾期和未逾期的比例的大小关系决定
   * 当前分组的比例 *大于* 整体比例时，$WOE$为正；即变量当前取值对判断个体是否会逾期起到的 *正向* 的影响
   * 当前分组的比例 *小于* 样本整体比例时，$WOE$为负；即变量当前取值对判断个体是否会逾期起到 *负向* 的影响
   * 当前分组的比例和整体比例 *相等* 时，$WOE$为0
  * $WOE$的取值范围是全体实数。
  
  
--------------------
> **$IV$的基本特点**

  * 对于变量的一个分组，这个分组的逾期和未逾期的比例与样本整体逾期和未逾期的比例相差越大，$IV$值越大；否则，$IV$值越小。
  * 极端情况下，当前分组的逾期和未逾期应的比例和样本整体的逾期和未逾期的比例相等时，$IV$值为0；
  * $IV$值的取值范围是$[0,+∞)$，且，当当前分组中只包含逾期用户或者未逾期用户时，$IV$ = $+∞$。

--------------------

> **$IV$值的选择**

| $IV$值大小 | 含义  
| :------:| :------: 
|  < 0.02 | 无预测能力 
|  0.02 ~ 0.1 | 较弱的预测能力 
|  0.1 ~ 0.3  | 预测能力一般 
|  0.3 ~ 0.5  | 较强的预测能力 
|  \> 0.5  | 可疑 
| $+∞$ 或 $-∞$ | 无意义 


--------------------

> **为什么用$IV$而不是直接用$WOE$**

  变量各分组的$WOE$和$IV$都隐含着这个分组对目标变量的预测能力这样的意义。那我们为什么不直接用$WOE$相加或者绝对值相加作为衡量一个变量整体预测能力的指标，而是要用$WOE$处理后的$IV$呢？原因有以下两点

* 当我们衡量一个变量的预测能力的时候，所使用的指标不应该是负数。基于此，乘以$(py_i-py_n)$这个系数，保证了变量每个分组的结果都是非负数。

  * 当一个分组的$WOE$是正数时，$(py_i-py_n)$也为正数
  * 当一个分组的$WOE$是负数时，$(py_i-py_n)$也为负数
  * 当一个分组的$WOE$是0时，$(py_i-py_n)$也为0


* 乘以$(py_i-py_n)$后，体现出了变量当前分组中个体的数量占整体个体数量的比例，也体现了变量预测能力的**大小**，以下表为例

| A | 逾期 | 不逾期 | 合计 | 逾期率 | $WOE$ | $IV$  |
| :------:| :------: | :------:| :------:| :------:| :------:| :------:|
| 1 | 90 | 10 | 100 | 90% | 4.394 | 0.03906  |
| 0 | 9910 | 89990 | 99900 | 10% | -0.0089 | 0.000079  |
| 合计 | 10000 | 90000 | 100000 | 10% | 4.4033 | 0.03914  |

  1. 从上表可以看出，当变量A取值1时，其逾期率达到了90%，非常的高，但是我们能否说变量A的预测能力非常强呢？不能。为什么呢？原因在于，A取1时，逾期率虽然很高，但这个分组的客户数太少了，占的比例太低了。虽然，如果一个客户在A这个变量上取1，那他有90%的响应可能性，但是一个客户变量A取1的可能性本身就非常的低。所以，对于样本整体来说，变量的预测能力并没有那么强。
  
  2. 从$WOE$和$IV$值的大小也可以看出，当A取1时，对应的$WOE$很高，但相应的$IV$却很低，原因就在于$IV$在$WOE$的前面乘以了一个系数$(py_i-py_n)$，而这个系数很好的考虑了这个分组中样本占整体样本的比例，比例越低，这个分组对变量整体预测能力的贡献越低。

  3. 总体而言，变量的预测能力强($WOE$大)，贡献大(群体占比高)，$IV$值才会高，如果只是预测能力强(即$WOE$高)，但是群体占比却很低，$IV$值同样会很低

In [21]:
import numpy as np

print 'woe_1:{:.4f}'.format(np.log((90*1.0/10000)/(10*1.0/90000)))
print 'iv_1:{:.7f}'.format(((90*1.0/10000)-(10*1.0/90000))*np.log((90*1.0/10000)/(10*1.0/90000)))

print 'woe_0:{:.4f}'.format(np.log((9910*1.0/10000)/(89990*1.0/90000)))
print 'iv_0:{:.7f}'.format(((9910*1.0/10000)-(89990*1.0/90000))*np.log((9910*1.0/10000)/(89990*1.0/90000)))

print 'IV:{:.5f}'.format(((90*1.0/10000)-(10*1.0/90000))*np.log((90*1.0/10000)/(10*1.0/90000))+((9910*1.0/10000)-(89990*1.0/90000))*np.log((9910*1.0/10000)/(89990*1.0/90000)))

woe_1:4.3944
iv_1:0.0390618
woe_0:-0.0089
iv_0:0.0000794
IV:0.03914


## $IV$的极端情况及处理方式

$IV$依赖$WOE$，并且$IV$是一个很好的衡量自变量对目标变量影响程度的指标。但是，使用过程中应该注意一个问题：变量的任何分组中，不应该出现逾期人数或未逾期人数为0的情况。

* 当变量的某个分组中，逾期人数为0，即 $\#y_i=0$ 时：$$WOE_i=ln(\frac{py_i}{pn_i})=ln(\frac{\frac{0}{\#y_T}}{\frac{\#n_i}{\#n_T}})=-∞$$  

* 当变量的某个分组中，未逾期人数为0，即 $\#n_i=0$ 时：$$WOE_i=ln(\frac{py_i}{pn_i})=ln(\frac{\frac{\#y_i}{\#y_T}}{\frac{0}{\#n_T}})=+∞$$  

当出现以上两种情况，不管$IV$值是正无穷还是负无穷，都是没有任何意义的，出现这种情况应该怎么做呢？

1. 如果可能，直接把这个分组做成一个规则，应该该分组对标的是全是好人或全是坏人。
2. 重新对目标变量进行分组，保证每个分组的逾期率即不为0，也不为100%
3. 人工干预，当某变量分组出现逾期人数或者未逾期人数为0的情况时，程序自动标注为1

# $WOE$值的其他应用

$WOE$编码之后，自变量其实具备了某种标准化的性质。也就是说，自变量内部的各个取值之间都可以直接进行比较（$WOE$之间的比较），而不同自变量之间的各种取值也可以通过$WOE$进行直接比较。

进一步地，可以研究自变量内部$WOE$值的变异（波动）情况，结合模型拟合出的系数，构造出各个自变量的贡献率及相对重要性。一般地，系数越大，$WOE$的方差越大，则自变量的贡献率越大（类似于某种方差贡献率）

# Python实现

In [ ]:
import numpy as np
from scipy import stats

def calciv_equal_percent(X_col,Y_col,k,point):
    a=[0 for i in range(k)]
    for i in range(k):
        a[i]=stats.scoreatpercentile(X_col, i*point) # point <= 100/(k-1) 
    woe=np.zeros(np.unique(a).shape)
    iv=np.zeros(np.unique(a).shape)
    # n_1表示逾期：逾期人数合计
    n_1 = np.sum(Y_col==1)
    n_1_group = np.zeros(np.unique(a).shape)
    # n_0表示未逾期：未逾期人数合计    
    n_0 = np.sum(Y_col==0)
    n_0_group = np.zeros(np.unique(a).shape)
    for i in range(len(np.unique(a))):
        if i < max(range(len(np.unique(a))-1)):
            n_1_group[i] = Y_col[(X_col >= a[i]) & (X_col < a[i+1]) & (Y_col == 1)].count()
            n_0_group[i] = Y_col[(X_col >= a[i]) & (X_col < a[i+1]) & (Y_col == 0)].count()
            woe[i] = np.log(((np.sum(n_1_group[i])/n_1)/(np.sum(n_0_group[i])/n_0)))
            iv[i] = (np.sum(n_1_group[i])/n_1-np.sum(n_0_group[i])/n_0)*np.log(((np.sum(n_1_group[i])/n_1)/(np.sum(n_0_group[i])/n_0)))
        elif i==len(np.unique(a))-1:
            n_1_group[i] = Y_col[(X_col >= a[i]) & (Y_col == 1)].count()
            n_0_group[i] = Y_col[(X_col >= a[i]) & (Y_col == 0)].count()
            woe[i] = np.log(((np.sum(n_1_group[i])/n_1)/(np.sum(n_0_group[i])/n_0)))
            iv[i] = (np.sum(n_1_group[i])/n_1-np.sum(n_0_group[i])/n_0)*np.log(((np.sum(n_1_group[i])/n_1)/(np.sum(n_0_group[i])/n_0)))
    IV=sum(iv)
    return  IV,woe

In [ ]:
#批量筛选需要手动设置cutpoint的变量
col_vars=['age']

# 初始化
k=5
point=10
col_iv_value=[]
col_woe_value=[]
for col in col_vars:
    X_col=data_set[col]
    col_iv_value.append(round(calciv_equal_percent(data_set[col],Y_col,k,point)[0],3))  
    col_woe_value.append(calciv_equal_percent(data_set[col],Y_col,k,point)[1])

# 计算变量woe编码的便准差
np.std(col_woe_value)

In [ ]:
# 用分箱后的woe值衍生

a=[0]*k  # k=5
for col in col_vars:
    for i in range(k):   # k=5
        a[i]=stats.scoreatpercentile(data_set[col], point*i)  # point =10
    for i in range(len(np.unique(a))):
        if i <= max(range(len(np.unique(a))-1)):
             data_set.loc[(data_set[col]>= a[i]) & (data_set[col]<=a[i+1]),col+'_woe']=calciv_equal_percent(data_set[col],Y_col,k,point)[1][i]
        elif i==max(range(len(np.unique(a)))):
             data_set.loc[data_set[col]>=a[i],col+'_woe']=calciv_equal_percent(data_set[col],Y_col,k,point)[1][i]

# 参考

1. 输公式：[点这里](http://www.domuse.com/markdown-and-latex-equation-handbook.html)
2. 做表格：[点这里](http://glgjing.github.io/blog/2015/04/03/markdown-biao-ge-yu-fa/)
3. 参考文献：[点这里](http://ucanalytics.com/blogs/information-value-and-weight-of-evidencebanking-case/)